In [1]:
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader 
from skimage.metrics import structural_similarity as ssim
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from matplotlib.lines import Line2D

from AE_functions import *
from eval_functions import *
from make_dataset import * 

%matplotlib qt

import warnings
warnings.filterwarnings("ignore")


In [2]:
## Define which training model, that should be evaluated

train = '53625'

In [3]:
## Plotting the reconstructions

no = [0, 0,50, 100, 250, 975] # Change which image, which have to be shown

fig, ax  = plt.subplots(2, 3, figsize=(12, 15))

for i in range(1,6):
    file = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/reconstruction{no[i]}.npy'
    x = np.load(file)

    x.astype(np.float64)

    # Normalize the image, so it fits the float32 format
    x = x[0,:,:]
    img_float32 = (x - np.min(x)) / (np.max(x) - np.min(x))

    ax[i//3, i%3].imshow(img_float32, cmap='gray')
    ax[i//3, i%3].set_title(f'Reconstructed Image, {no[i]} epochs')


file = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/{train}_training/original.npy'
x = np.load(file)
x.astype(np.float64)
x = x[0,:,:]
img_float32 = (x - np.min(x)) / (np.max(x) - np.min(x))

ax[0,0].imshow(img_float32, cmap='gray')
ax[0,0].set_title('Original Image')


fig.show()

fig.savefig(f'C:/Users/julie/OneDrive/Skrivebord/merged_{train}.png')  # Save the plot as a PNG file

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/53625_training/reconstruction0.npy'

In [3]:
## Plotting losses from training

file_loss = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/A_Trainings/{train}_training/o_loss.npy'
file_val_loss = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/A_Trainings/{train}_training/Val_loss.npy'

o_loss = np.load(file_loss)
val_loss = np.load(file_val_loss)
print(f"Converged towards {np.mean(o_loss[250:])}")

fig, ax = plt.subplots()
ax.set_title(f'Model Loss, batch_size=780, lr=0.001, wd=0.0005')
ax.set_xlabel('Epoch')
ax.set_ylabel('Avg. loss')
# ax.set_xticks(np.arange(0, len(o_loss), step= 100))
ax.set_xticks(np.arange(0, len(o_loss), step= 200))

ax.plot(list(range(1, len(o_loss)+1, 1)), o_loss, label='Training loss', color='b')  # Update the plot with the current loss
# ax.plot(list(range(25, len(o_loss)+1, 25)), val_loss, label='Validation loss', color='r')
# ax.plot(list(range(3, len(o_loss)+1, 4)), val_loss[:-62], label='Validation loss', color='r')
ax.plot(list(range(20, len(o_loss)+1, 20)), val_loss, label='Validation loss', color='r')

ax.legend()
plt.show()
fig.savefig(f'C:/Users/julie/OneDrive/Skrivebord/model_loss_{train}.png')  # Save the plot as a PNG file

Converged towards 0.00016877190224366152


_____________________________

In [1]:
## Loading test image + outlier image

# Loading data (testset, 41 elements)
img_dir_test = "C:/Users/julie/Bachelor_data/crops_validation_prep/img"
heatmap_dir_test = "C:/Users/julie/Bachelor_data/crops_validation_prep/heatmaps"
msk_dir_test = "C:/Users/julie/Bachelor_data/crops_validation_prep/msk"

VerSe_test = LoadData(img_dir=img_dir_test, msk_dir = msk_dir_test, distfield_dir=heatmap_dir_test)
test_loader = DataLoader(VerSe_test, batch_size=1, shuffle=False, num_workers=0)

healthy, outlier = healthy_outlier(test_loader)
print(len(healthy), len(outlier))
# Create datasets
dataset_healthy = generate_dataset(healthy, 120)
dataset_outlier = generate_dataset_outlier(outlier, 120, radius=30, mode="")


NameError: name 'LoadData' is not defined

In [8]:
plt.imshow(dataset_healthy[0][0], cmap='gray')
plt.show()

In [4]:
## Evaluating the model with new data (healthy and outliers)

# Loading the model
model_path = f'C:/Users/julie/OneDrive/Skrivebord/Bachelor/JK_bachelor/A_Trainings/{train}_training/model_conv_1999.pth'

model = load_model(model_path, [1, 8, 16, 32, 64])


# Testing the model with MSE
error_healthy = []
error_outlier = []

for i in range(len(dataset_healthy)):
    # test_healthy, _, _  = test_loader.dataset[i]
    test_healthy = dataset_healthy[i]
    loss = evaluate_model(model, test_healthy[0].unsqueeze(dim=0))
    error_healthy.append(loss)

    # test_outlier, _, _ = test_loader_o.dataset[i]
    test_outlier = dataset_outlier[i]
    loss = evaluate_model(model, test_outlier[0].unsqueeze(dim=0))
    error_outlier.append(loss)

-----

# MSE

In [5]:
## Make histogram


hist_values1, hist_values2, bins1, bins2 = plot_histograms(error_healthy, error_outlier, 10)

mean_healthy = np.mean(error_healthy)
std_healthy = np.std(error_healthy)

mean_outlier = np.mean(error_outlier)
std_outlier = np.std(error_outlier)

In [6]:
print(f"Mean healthy: {mean_healthy}, std healthy: {std_healthy}")
print(f"Mean outlier: {mean_outlier}, std outlier: {std_outlier}")

Mean healthy: 0.0004540509785632215, std healthy: 0.00047075337899920113
Mean outlier: 0.0007210962678072975, std outlier: 0.0007537231888396763


In [7]:
## Plotting the difference from outlier input to reconstructions
org_img = dataset_outlier[0].squeeze()
reconstructed_img = model(dataset_outlier[0].unsqueeze(dim=0)).squeeze().detach().numpy()

fig, ax = plt.subplots(1, 3, figsize=(12, 4))

ax[0].imshow(dataset_outlier[0].squeeze(), cmap='gray')
ax[0].set_title('Original Image')

ax[1].imshow(reconstructed_img, cmap='gray')
ax[1].set_title('Reconstructed Image')

ax[2].imshow(org_img - reconstructed_img, cmap='bwr')
ax[2].set_title('Difference')
fig.colorbar(ax[2].imshow(org_img - reconstructed_img, cmap='bwr'))

plt.show()


------

# SSIM

In [8]:
## Evaluating the model with new data (healthy and outliers)

# Testing the model
ssim_healthy = []
ssim_outlier = []

for i in range(len(dataset_healthy)):
    # test_healthy, _, _  = test_loader.dataset[i]
    test_healthy = dataset_healthy[i]
    loss = evaluate_model_SSIM(model, test_healthy[0].unsqueeze(dim=0))
    ssim_healthy.append(loss)

    # test_outlier, _, _ = test_loader_o.dataset[i]
    test_outlier = dataset_outlier[i]
    loss = evaluate_model_SSIM(model, test_outlier[0].unsqueeze(dim=0))
    ssim_outlier.append(loss)

hist_values1, hist_values2, bins1, bins2 = plot_histograms_SSIM(ssim_healthy, ssim_outlier, 15)

In [9]:
## Plotting the difference from outlier input to reconstructions

org_img = dataset_outlier[0].squeeze()
org_img_np = org_img.cpu().detach().numpy()

reconstructed_img = model(dataset_outlier[0].unsqueeze(dim=0)).squeeze().detach().numpy()
reconstructed_img_np = reconstructed_img.astype(np.float64)


# Compute SSIM
mssim, ssim_map  = ssim(org_img_np, reconstructed_img_np, full=True) # full=True returns the full structural similarity image
print('The mean structural similarity index over the image is:',mssim)
#plot
fig, ax = plt.subplots(1, 3, figsize=(12, 4))

ax[0].imshow(org_img_np, cmap='gray')
ax[0].set_title('Original Image')

ax[1].imshow(reconstructed_img_np, cmap='gray')
ax[1].set_title('Reconstructed Image')

ax[2].imshow(ssim_map, cmap='bwr')
ax[2].set_title('SSIM Map')
fig.colorbar(ax[2].imshow(ssim_map, cmap='bwr'))

plt.show()

The mean structural similarity index over the image is: 0.9831384337692096


-----

# Latent Space

In [10]:
# Collect latent representations for healthy and outlier data
latent_representations_healthy = collect_latent_representations(model, dataset_healthy)
latent_representations_outlier = collect_latent_representations(model, dataset_outlier)
# print(latent_representations_healthy[0].shape)


# Flatten the collected latent representations
latent_representations_healthy_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_healthy], axis=0)
latent_representations_outlier_flat = np.concatenate([latent.flatten().reshape(1, -1) for latent in latent_representations_outlier], axis=0)

merged = np.concatenate((latent_representations_healthy_flat, latent_representations_outlier_flat), axis=0)
target = np.zeros((merged.shape[0], 1))
target[120:] = 1

target2 = []
for i in range(merged.shape[0]):
    if i < 120:
        target2.append('Healthy')
    else:
        target2.append('Outlier')
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target2)


-----

# PCA

In [11]:
## Explained Variance

pca = PCA().fit(merged)
print(np.cumsum(pca.explained_variance_ratio_))


plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.title('PCA explained variance')
plt.grid()
plt.show()


[0.18953142 0.25426736 0.30560464 0.34805405 0.38748682 0.41621858
 0.44307014 0.46698776 0.4866832  0.5055224  0.52170014 0.535875
 0.5495374  0.5624044  0.57505035 0.5863494  0.5974953  0.6077024
 0.6172996  0.6266624  0.6355506  0.64380044 0.65193427 0.6597651
 0.6671287  0.6740743  0.6807611  0.6873211  0.69382393 0.69991297
 0.70593065 0.7115652  0.71702725 0.7223506  0.7275716  0.73254293
 0.7373642  0.74208176 0.7466772  0.75112754 0.75552243 0.7597821
 0.76395124 0.76795506 0.7717683  0.7755602  0.7792517  0.7828778
 0.7864152  0.78993595 0.7934135  0.79677117 0.8000928  0.80330485
 0.8064563  0.809516   0.8124417  0.8153263  0.818142   0.8209096
 0.8236459  0.8263357  0.8289593  0.83157396 0.8341256  0.8366277
 0.8391174  0.84150594 0.84385043 0.84614354 0.8483829  0.85059834
 0.8527861  0.85490525 0.8569895  0.85906714 0.86109704 0.8630902
 0.8650403  0.86695516 0.86885655 0.8707203  0.87257105 0.8743971
 0.87618697 0.87794137 0.87966895 0.8813872  0.88307035 0.88474375
 0.88

In [13]:

pca = PCA(n_components=2) 
# pca.fit(merged)
pca.fit(latent_representations_healthy_flat)

# Transform
latent_pca = pca.transform(merged)


## Plotting the PCA visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

plt.figure(figsize=(10, 6))
scatter = plt.scatter(latent_pca[:, 0], latent_pca[:, 1], c=colors, alpha=0.5) #, cmap='viridis')

plt.title('PCA Visualization of Latent Space Representations')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

plt.show()

In [14]:
# Perform PCA with 3 components
pca = PCA(n_components=3)
pca.fit(merged)
# pca.fit(latent_representations_healthy_flat)

#Transform
latent_pca = pca.transform(merged)

## Plotting the PCA visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

# Plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latent_pca[:, 0], latent_pca[:, 1], latent_pca[:, 2], c=colors, alpha=0.5)

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

ax.set_title('PCA Visualization of Latent Space Representations (3D)')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
ax.legend(handles=legend_elements)
plt.show()

----

# TSNE

In [15]:
tsne = TSNE(n_components=2, random_state=42, perplexity=40, init='pca', learning_rate=100, n_iter=10000)

# Transform
latent_tsne = tsne.fit_transform(merged)

## Plotting the t-SNE visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

plt.figure(figsize=(10, 6))
scatter = plt.scatter(latent_tsne[:, 0], latent_tsne[:, 1], c=colors, alpha=0.5) #, cmap='viridis')

plt.title('TSNE Visualization of Latent Space Representations')
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

plt.show()


In [16]:
# Perform tsne with 3 components
tsne = TSNE(n_components=3, random_state=42, perplexity=30, learning_rate=100) 

# Transform
latent_tsne = tsne.fit_transform(merged)

## Plotting the t-SNE visualization
color_mapping = {'Healthy': 'blue', 'Outlier': 'red'}
colors = [color_mapping[label] for label in target2]

# Plot
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(latent_tsne[:, 0], latent_tsne[:, 1], latent_tsne[:, 2], c=colors, alpha=0.5)

legend_elements = [Line2D([0], [0], marker='o', color='w', label='Healthy', markersize=10, markerfacecolor='blue'),
                   Line2D([0], [0], marker='o', color='w', label='Outlier', markersize=10, markerfacecolor='red')]
plt.legend(handles=legend_elements)

ax.set_title('TSNE Visualization of Latent Space Representations (3D)')
ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_zlabel('t-SNE 3')
ax.legend(handles=legend_elements)
plt.show()